In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
orders = pd.read_csv('orders.csv', sep=';')

In [3]:
items = pd.read_csv('items.csv', sep=';')

## Orders EDA

In [4]:
orders.head()

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
0,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S101,1,NaN,3.506048,0.000000,3.496395,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
1,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S113,1,NaN,-0.705913,0.000000,-0.717209,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
2,2000093388,2020-04-24 00:00:00,COD,ZB00089178,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
3,2000093388,2020-04-24 00:00:00,COD,ZB00138060,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
4,2000093388,2020-04-24 00:00:00,COD,ZB00015664,1,NaN,19.379845,18.731008,32.015504,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00


In [5]:
orders.shape

(2128524, 14)

In [6]:
orders.describe()

,order_id,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl
count,2.128524e+06,2.128524e+06,10390.000000,2.128524e+06,2.128524e+06,2.128426e+06
mean,2.039053e+09,1.029551e+00,1.000192,3.102203e+01,1.903846e+01,3.942803e+01
std,7.691450e+07,5.852888e-01,0.019621,4.066967e+01,2.665736e+01,5.013470e+01
min,2.000000e+07,-1.000000e+00,1.000000,-1.859721e+00,0.000000e+00,-1.771163e+00
25%,2.000112e+09,1.000000e+00,1.000000,2.551573e+00,0.000000e+00,2.613605e+00
50%,2.100026e+09,1.000000e+00,1.000000,1.442870e+01,7.829457e+00,2.034849e+01
75%,2.100322e+09,1.000000e+00,1.000000,4.481395e+01,2.871260e+01,5.810465e+01
max,2.200024e+09,1.360000e+02,3.000000,7.203857e+02,2.440504e+03,7.444543e+02


In [7]:
orders.isnull().sum()

order_id                     0
date                         0
payment                    567
item_code                    0
quantity                     0
gift_quantity          2118134
unit_price_vat_excl          0
unit_cogs                    0
unit_rrp_vat_excl           98
department                   0
channel                      0
owner                        0
site                         0
CreatedAt                    0
dtype: int64

##### Must look into why there are null values in payment column
##### Must look into why there can be negative unit price and RRP

In [8]:
print(f"Order date range: {orders['date'].min()} to {orders['date'].max()}")

Order date range: 2019-06-01 00:00:00 to 2022-01-14 00:00:00


In [31]:
for col in orders:
    print(f'{orders[col].value_counts()} \n')

2100248254    76
2100068062    53
2000271974    50
2100066795    46
2000187597    44
              ..
2100190797     1
2100190796     1
2100190789     1
2100190786     1
521011438      1
Name: order_id, Length: 925050, dtype: int64 

2021-11-28 00:00:00    8219
2021-12-13 00:00:00    7801
2021-12-14 00:00:00    7536
2021-12-12 00:00:00    7518
2021-12-15 00:00:00    7287
                       ... 
2019-06-03 00:00:00      50
2019-06-02 00:00:00       9
2019-06-16 00:00:00       6
2019-06-15 00:00:00       5
2019-06-08 00:00:00       4
Name: date, Length: 958, dtype: int64 

COD                    1239267
GOPAY_CARD              253689
PAYPAL                  226131
CARD_GOPAY              198801
CARD_CSOB               115246
CHECKOUT_CARD            84398
PAYPAL_EXPRESS            5322
CHECKOUT_APPLE            2626
CHECKOUT_IDEAL            1388
CHECKOUT_BANCONTACT        551
BANK_TRANSFER              356
CASH                        73
ZALOHA                      32
CSOB_CARD      

##### Must look into most frequent item_codes (S101, S113, S114)
##### Must look into difference between channel and owner
##### Department column does not contain meaningful information, can be dropped
##### Must look into what it means when "site" is Other

In [37]:
filter = orders['payment'].isna()
# orders[filter]
orders[filter]['date'].value_counts()

2022-01-14 00:00:00    563
2021-07-12 00:00:00      1
2021-07-26 00:00:00      1
2021-07-27 00:00:00      1
2021-07-29 00:00:00      1
Name: date, dtype: int64

##### Can drop these rows, majority of nulls occurring on one day suggests payment processor outage

In [11]:
filter = orders['item_code']=='S113'
orders[filter]

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
1,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S113,1,NaN,-0.705913,0.0,-0.717209,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
7,2000093388,2020-04-24 00:00:00,COD,S113,1,NaN,0.929070,0.0,0.930233,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
10,2000093389,2020-04-24 00:00:00,COD,S113,1,NaN,0.929070,0.0,0.930233,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:50:00
13,2000093390,2020-04-24 00:00:00,CARD_CSOB,S113,1,NaN,-0.640698,0.0,-0.658915,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:50:00
16,2000093391,2020-04-24 00:00:00,COD,S113,1,NaN,0.843585,0.0,0.843585,E-COMMER,FTB.skC,FTB.skO,11teamsports.sk,2020-04-24 21:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128448,2200024140,2022-01-14 00:00:00,NaN,S113,1,NaN,0.748969,0.0,0.748969,E-COMMER,FIT.skC,FIT.skO,top4fitness.sk,2022-01-14 10:55:00
2128451,2200024142,2022-01-14 00:00:00,NaN,S113,1,NaN,0.929070,0.0,0.930233,E-COMMER,RUN.czC,RUN.czO,top4running.cz,2022-01-14 10:55:00
2128455,2200024143,2022-01-14 00:00:00,NaN,S113,1,NaN,0.805326,0.0,0.766977,E-COMMER,11TS.roC,11TS.roO,11teamsports.ro,2022-01-14 11:00:00
2128461,2200024147,2022-01-14 00:00:00,NaN,S113,1,NaN,0.805326,0.0,0.766977,E-COMMER,11TS.roC,11TS.roO,11teamsports.ro,2022-01-14 11:00:00


In [41]:
filter = orders['unit_price_vat_excl'] < 0
orders[filter]

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
1,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S113,1,NaN,-0.705913,0.0,-0.717209,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
13,2000093390,2020-04-24 00:00:00,CARD_CSOB,S113,1,NaN,-0.640698,0.0,-0.658915,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:50:00
20,2000093392,2020-04-24 00:00:00,CARD_CSOB,S113,1,NaN,-0.640698,0.0,-0.658915,E-COMMER,Sport.cC,Sport.cO,top4sport.cz,2020-04-24 21:55:00
23,2000093393,2020-04-24 00:00:00,CARD_CSOB,S113,1,NaN,-0.640698,0.0,-0.658915,E-COMMER,Sport.cC,Sport.cO,top4sport.cz,2020-04-24 21:55:00
34,2000093396,2020-04-24 00:00:00,CARD_CSOB,S113,1,NaN,-0.640698,0.0,-0.658915,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238959,2100122082,2021-03-25 00:00:00,CARD_GOPAY,S113,1,NaN,-0.426593,0.0,-0.436750,E-COMMER,FTB.deC,FTB.deO,top4football.de,2021-03-25 08:20:00
1239175,2100122192,2021-03-25 00:00:00,CARD_GOPAY,S113,1,NaN,-0.426593,0.0,-0.436750,E-COMMER,RUN.deC,RUN.deO,top4running.de,2021-03-25 10:05:00
1239439,2100122343,2021-03-25 00:00:00,GOPAY_CARD,S113,1,NaN,-0.872721,0.0,-0.831163,E-COMMER,FTB.roC,FTB.roO,11teamsports.ro,2021-03-25 11:35:00
1239451,2100122353,2021-03-25 00:00:00,GOPAY_CARD,S113,1,NaN,-0.872721,0.0,-0.831163,E-COMMER,FTB.roC,FTB.roO,11teamsports.ro,2021-03-25 11:40:00


##### These appear to be returns

## Items EDA

In [13]:
items.head()

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
0,ZB00210807,Studio Metallic LS Top,51951501,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,fitness,women,adults,Black,L
1,ZB00210813,TRAIN TECH EVOKNIT SS TEE,52011101,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Black,S
2,ZB00210815,TRAIN TECH EVOKNIT SS TEE,52011123,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Orange,L
3,ZB00210821,TRAIN TECH EVOKNIT SS TEE,52011130,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Yellow,XL
4,ZB00261295,LIGA Baselayer Tee LS,655920-027,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,football,men,adults,Green,L


In [14]:
items.shape

(309128, 16)

In [15]:
items.isnull().sum()

item_code         0
item_name      3126
style          6797
brand_id          0
name              0
group0_id        35
group0          826
group1_id      7419
group1        43780
group2_id      7419
group2       237355
category      39846
gender         7484
age            7566
color         45596
size           8041
dtype: int64

##### Many nulls, item_name and style are the most important to look into

In [16]:
items['item_name'].value_counts()

DUPLICITNÍ ARTIKL                         427
UA Classic Slip-On                        397
GEL-NIMBUS 23                             343
React Infinity Run Flyknit 2              328
NIKE AIR ZOOM PEGASUS 36                  323
                                         ... 
Under Armour AF Graphic                     1
FFF Strike                                  1
U NK HERTHA BSC HOME DRY SOCKS 2020/21      1
Borussia Dortmund Away 2019/20              1
iPhone 7 32GB Black                         1
Name: item_name, Length: 33788, dtype: int64

In [42]:
items['style'].value_counts()

DUPLICITNÍ ARTIKL    186
907564-017            31
vn000d5invy1          30
aq4184-010            30
907566-017            27
                    ... 
928983-010             1
e099ha00               1
1326799-486            1
365340-002             1
ZB00025686             1
Name: style, Length: 65200, dtype: int64

In [17]:
filter = items['item_code']=='S101'
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
306868,S101,Dopravné,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### S101 Dopravné translates to Freight. 

#### žádný výrobce translates to no manufacturer.

#### S113 Platba translates to Payment

#### S114 Sleva translates to Discount

In [18]:
filter = items['item_code']=='S113'
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
307919,S113,Platba,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
filter = items['item_code']=='S114'
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
308992,S114,Sleva,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
filter = items['item_code']=='ZB00077277'
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
285821,ZB00077277,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,M


In [21]:
filter = items['item_code']=='ZB00053597'
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
269154,ZB00053597,Nike Everyday 3 pack,sx7676-100,84,Nike,200.0,Apparel,290.0,Socks,0.0,NaN,football,men,adults,White,L


In [22]:
filter = (items['item_name']=='GEL-NIMBUS 23')
items[filter].head(50)

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
72904,ZB00186782,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73749,ZB00186781,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74974,ZB00186776,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74975,ZB00186778,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75438,ZB00186777,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75479,ZB00186773,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75498,ZB00186775,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75619,ZB00186779,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75667,ZB00186772,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75822,ZB00186780,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
filter = (items['item_name']=='GEL-NIMBUS 23') & (items['size'].isna())
items[filter].head(100)

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
72904,ZB00186782,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73749,ZB00186781,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74974,ZB00186776,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74975,ZB00186778,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75438,ZB00186777,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141056,ZB00186361,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141099,ZB00186465,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141324,ZB00186352,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141480,ZB00186474,GEL-NIMBUS 23,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
filter = items['group0_id']==999
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
1410,ZB00151257,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,47
1411,ZB00151259,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,46
1660,ZB00220924,erima sport bag club 5 gr s,723333,132,Erima,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,L
2243,ZB00227818,TE Piping Tracksuit,gs9309,88,Reebok,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,XXL
2303,ZB00151226,ftblPLAY Training Pant Puma Black,65593301,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309123,ZB00073434,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309124,ZB00075104,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309125,ZB00073523,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309126,ZB00165469,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
filter = items['brand_id']==-1
items[filter]

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
12505,ZB00000222,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12661,ZB00000224,"Multifunkcní batoh Hamelin, cerná",NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13255,ZB00000221,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16808,ZB00000223,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50614,ZB00186641,DYNABLAST,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309123,ZB00073434,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309124,ZB00075104,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309125,ZB00073523,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309126,ZB00165469,NaN,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
filter = (items['item_name']=='Xfer Montecarlo')
items[filter].head(50)

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
12505,ZB00000222,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13255,ZB00000221,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16808,ZB00000223,Xfer Montecarlo,NaN,-1,- žádný výrobce -,999.0,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
filter = orders['item_code']=='ZB00000223'
orders[filter]

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
